pip install pickle5

In [4]:
pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 10.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [5]:

import gc
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import seaborn as sns
import pandas as pd
from collections import Counter
from heapq import nlargest





In [6]:
df_pol_train = pl.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
df_pol_test = pl.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

In [8]:

#sample_df = df_test_json[:150000]
clicks_article_list = []
carts_article_list = []
orders_article_list = []
    
i = 0    
for type in df_pol_train["type"]:
        if type == 0:
                clicks_article_list.append(df_pol_train["aid"][i])
        elif type == 1:
                carts_article_list.append(df_pol_train["aid"][i])
        else:
                orders_article_list.append(df_pol_train["aid"][i])

        i += 1

In [9]:
# Create dictionaries with articles and their frequencies
article_click_freq = Counter(clicks_article_list)
article_carts_freq = Counter(carts_article_list)
article_order_freq = Counter(orders_article_list)

In [10]:
top_click_article = sorted(article_click_freq, key=article_click_freq.get, reverse=True)[:20]
top_carts_article = sorted(article_carts_freq, key=article_carts_freq.get, reverse=True)[:20]
top_order_article = sorted(article_order_freq, key=article_order_freq.get, reverse=True)[:20]

In [11]:
sorted(article_click_freq, key=article_click_freq.get, reverse=True)[:20]

[1460571,
 108125,
 29735,
 485256,
 1733943,
 184976,
 832192,
 1502122,
 554660,
 1603001,
 986164,
 166037,
 322370,
 1236775,
 231487,
 959208,
 332654,
 1196256,
 95488,
 620545]

In [12]:
# Create a dict with this info
frequent_articles = {'clicks': top_click_article, 'carts':top_carts_article, 'order':top_order_article}

In [13]:

r_total = []

def calc_recall(pred_clicks, pred_carts, pred_orders, test_clicks, test_carts, test_orders):
    
    if set(pred_clicks).intersection(set(test_clicks)):
        r_click = 1
    else:
        r_click = 0
    if set(pred_carts).intersection(set(test_carts)):
        r_carts = 1
    else:
        r_carts = 0
    if set(pred_orders).intersection(set(test_orders)):
        r_order = 1
    else:
        r_order = 0
    
    
    r_session = 0.1*r_click + 0.3*r_carts + 0.6*r_order

    r_total.append(r_session)
    


In [27]:

for j in range(len(sample_test_df)):
    calc_recall(top_click_article, top_carts_article, top_order_article, sample_test_df["test_clicks"][j],sample_test_df["test_carts"][j],sample_test_df["test_orders"][j])
  
result = sum(r_total)/len(r_total)
print(result)

0.007


In [17]:
submission = pd.read_csv("/kaggle/input/otto-recommender-system/sample_submission.csv")

In [18]:
submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [19]:
submission['labels'] = submission['labels'].apply(lambda x: top_click_article )


In [20]:
submission.head()

,session_type,labels
0,12899779_clicks,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
1,12899779_carts,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
2,12899779_orders,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
3,12899780_clicks,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
4,12899780_carts,"[1460571, 108125, 29735, 485256, 1733943, 1849..."


In [21]:
top_clicks_submission = ' '.join(str(i) for i in top_click_article)
top_carts_submission = ' '.join(str(i) for i in top_carts_article)
top_order_submission = ' '.join(str(i) for i in top_order_article)


In [22]:
def check_type(x):
    if "click" in x:
        return top_clicks_submission
    elif "carts" in x:
        return top_carts_submission
    else:
        return top_order_submission

In [23]:
submission['labels'] = submission['session_type'].apply(check_type )

In [24]:
submission.head()

,session_type,labels
0,12899779_clicks,1460571 108125 29735 485256 1733943 184976 832...
1,12899779_carts,485256 152547 33343 166037 1733943 231487 2973...
2,12899779_orders,231487 166037 1733943 1445562 1022566 801774 1...
3,12899780_clicks,1460571 108125 29735 485256 1733943 184976 832...
4,12899780_carts,485256 152547 33343 166037 1733943 231487 2973...


In [25]:
submission.to_csv('submission.csv', index=False)